<a href="https://colab.research.google.com/github/MaiMejia/ML-Projects/blob/main/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> Bitcoin Trading Agent - Backtesting</b>

##### <b>Context</b>

<b>Background:</b>
<br>We are a fintech company focused on cryptocurrency trading building a smart bitcoin trading system designed to operate with minimal human supervision and continuously adapt to changing market conditions. The agent must dynamically manage budget allocation, shift between strategies, and make autonomous trading decisions while running 24/7.

This project will give you experience with real-time algorithmic trading, feature engineering in a volatile domain, LLM-assisted decision-making, and deploying robust, cloud-based AI systems that bridge automation with finance.

<b>Project Objectives:</b>
- Accept a configurable budget (e.g., $1K - 100K)
- Use Dollar-Cost Averaging (DCA) to accumulate more bitcoin when prices drop, distributing buys over time or price levels
- Implement an ATR-based stop-loss strategy to manage short-term trades and avoid excessive loss exposure
- Switch between different strategies (e.g., day trading, swing trading, value investing)
- Adapt continuously to market conditions, ideally with the help of a lightweight LLM
- Run 24/7 and deploy in a cloud environment
- Send Telegram notifications for each trade made
- Send a weekly email report every Monday at 9:00AM via Gmail


##### <b>1. Installing and loading libraries</b>

In [ ]:
# --- 0. AUTOMATIC LIBRARY INSTALLATION ---
import os
import sys
import subprocess
# --------------------------------------------------

REQUIRED_LIBRARIES = [
    ('pandas-ta', 'pandas_ta'),
    ('yfinance', 'yfinance'),
    ('numpy', 'numpy'),              # PyPI: numpy, Import: numpy
    ('pandas', 'pandas'),            # PyPI: pandas, Import: pandas
    ('scikit-learn', 'sklearn'),    # PyPI: scikit-learn, Import: sklearn
    ('gspread', 'gspread'),
    ('oauth2client', 'oauth2client'),
]

def install_libraries(libraries):
    """Installs missing Python libraries using pip."""
    print("Checking for required libraries...")

    for pypi_name, import_name in libraries:
        try:
            # Check for the *import* name
            __import__(import_name)
        except ImportError:
            print(f"'{import_name}' not found. Attempting to install '{pypi_name}'...")
            try:
                # Install using the *PyPI* name
                subprocess.check_call([sys.executable, "-m", "pip", "install", pypi_name, '--quiet'])
                print(f"Successfully installed {pypi_name}.")
            except subprocess.CalledProcessError as e:
                print(f"Error installing {pypi_name}: {e}")


install_libraries(REQUIRED_LIBRARIES)

Checking for required libraries...
'pandas_ta' not found. Attempting to install 'pandas-ta'...
'sklearn' not found. Attempting to install 'scikit-learn'...


In [ ]:

# Imports follow the installation block
import numpy as np
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Tuple, Optional
import time # Added for potential delays
import requests

import yfinance as yf
import json
import pandas_ta

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### <b>Data Retrival</b>

In [ ]:
## -------------------------------------------------------- ##
# ------------- DATA RETRIEVAL  -------
## -------------------------------------------------------- ##

# --- CONFIGURATION (Change these values to adjust the data) ---
TICKER = 'BTC-USD'
HISTORY_START_DATE = '2023-01-01' # Fetch the last 2 years of data
HISTORY_INTERVAL = '1d'

# Define where to save the file
SAVE_FOLDER = '/content/drive/MyDrive/ColabNotebooks/Proj5/data_btc'
FULL_PATH = os.path.join(SAVE_FOLDER, "btcdata.csv")


def fetch_and_save_data(ticker, start_date, interval, save_path):
    """
    Fetches historical data for the configured Ticker and saves it to a CSV file.
    """

    try:
        # Use yfinance.download for the simplest fetching method
        data = yf.download(
            tickers=ticker,
            start=start_date,
            interval=interval
        )

        if data.empty:
            print("❌ Error: No data returned from yfinance.")
            return

        if isinstance(data.columns, pd.MultiIndex):
              # Flatten the MultiIndex to a single index of (Price_Type, Ticker) tuples
              data.columns = data.columns.to_flat_index()
              # Convert the tuples to cleaner string names like 'Close_BTC-USD'
              # You can customize this, but for a single ticker, the first element is enough
              data.columns = [col[0] for col in data.columns]
        else:
              data.columns = [col for col in data.columns]

        # Clean up columns: Rename 'Adj Close' to 'Close' and drop redundant columns
        data = data.rename(columns={"Adj Close": "Close"}, errors='ignore')
        df_final = data.drop(columns=['Dividends', 'Stock Splits', 'Repaired?'], errors='ignore')

        # 2. Prepare the save location
        os.makedirs(SAVE_FOLDER, exist_ok=True)

        # 3. Save the final dataset (overwrites the file every time)
        df_final.to_csv(save_path, index=True)

    except Exception as e:
            print(f"❌ A critical error occurred during fetching or saving: {e}")


# --- GETTING THE DATASET ---
fetch_and_save_data(
    TICKER,
    HISTORY_START_DATE,
    HISTORY_INTERVAL,
    FULL_PATH
)

[*********************100%***********************]  1 of 1 completed


##### <b>Data Processing</b>

In [ ]:
## -------------------------------------------------------------- ##
# -------------  DATA PROCESSING & ADDING TA INDICATORS   -------
## -------------------------------------------------------------- ##


# --- CONFIGURATION (Ensure these paths match your retrieval script) ---
SAVE_FOLDER = '/content/drive/MyDrive/ColabNotebooks/Proj5/data_btc'
RAW_PATH = os.path.join(SAVE_FOLDER, "btcdata.csv")
PROCESSED_FILE = os.path.join(SAVE_FOLDER, "btc_procdata.csv")

def process_data_and_add_features():
    """
    Loads raw data, adds essential features (EMA, RSI, ATR, MACD, Lag),
    drops NaNs, and saves the final DataFrame, ensuring all strategy inputs are present.
    """

    if not os.path.exists(RAW_PATH):
        print(f"❌ Error: Raw data file not found at {RAW_PATH}.")
        return

    # 1. Load the raw data
    df = pd.read_csv(RAW_PATH, index_col=0, parse_dates=True)

    # Coerce price columns to numeric and drop rows where price data is missing
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=numeric_cols, inplace=True)

    original_length = len(df)

    # 2. Add ALL KEY Technical Analysis Indicators using pandas-ta lib
    df.ta.ema(length=20, append=True)                # EMA (Exponential Moving Average) - Adds 'EMA_20'
    df.ta.rsi(length=14, append=True)                # RSI (Relative Strength Index) - Adds 'RSI_14'
    df.ta.atr(length=14, append=True)                # ATR (Average True Range) - Adds 'ATR_14'

    # MACD (Moving Average Convergence Divergence) - Adds 'MACD_12_26_9', 'MACDH_12_26_9', 'MACDS_12_26_9'
    df.ta.macd(append=True)                           # MACD is crucial for the SWING_TRADE strategy logic
    df['Close_Lag_1'] = df['Close'].shift(1)          # LAG Add Lagged Feature

    # 3. Clean up and Save
    df.dropna(inplace=True)   # Drop rows that are NaN due to the indicator lookback period

    # RENAME COLUMNS FOR SIMPLICITY in the trading script (matching your expected variables)
    df.rename(columns={
        'ATRr_14': 'ATR',
        'RSI_14': 'RSI',
        'EMA_20': 'EMA_20',
        'MACD_12_26_9': 'MACD',
        'MACDs_12_26_9': 'MACD_Signal'
        # MACDH is the Histogram, not directly used in the switch logic, so we omit renaming it
    }, inplace=True)

    os.makedirs(SAVE_FOLDER, exist_ok=True)    # Ensure folder exists
    df.to_csv(PROCESSED_FILE, index=True)     # Save the processed data

    print(f"✅ Data processed and saved to {PROCESSED_FILE}.")
    print(f"   Indicators added: RSI, ATR, EMA_20, MACD, MACD_Signal.")
    print(f"   Final Rows: {len(df)} (Dropped {original_length - len(df)} rows for lookback)")
    # print(df.tail(1)[['Close', 'RSI', 'MACD', 'MACD_Signal', 'ATR', 'EMA_20']])

process_data_and_add_features()

✅ Data processed and saved to /content/drive/MyDrive/ColabNotebooks/Proj5/data_btc/btc_procdata.csv.
   Indicators added: RSI, ATR, EMA_20, MACD, MACD_Signal.
   Final Rows: 1006 (Dropped 33 rows for lookback)


##### <b>Loading the TA and Sentiment Data</b>

In [ ]:
## ---------------------------------------------------------------- ##
# 🚀 BLOCK 1: LOAD AND PREPARE ALL RAW/EXTERNAL DATA (TA & FGI)
# --- Includes Date Filtering for Backtesting ---
## ---------------------------------------------------------------- ##


# --- CONFIGURATION (Ensure this aligns with your Colab drive) ---
SAVE_FOLDER = '/content/drive/MyDrive/ColabNotebooks/Proj5/data_btc'
PROCESSED_FILE = os.path.join(SAVE_FOLDER, "btc_procdata.csv")
CLEANED_TA_FILE = os.path.join(SAVE_FOLDER, "btc_cleaned_ta_data.csv") # Output 1
FGI_PROCESSED_FILE = os.path.join(SAVE_FOLDER, "btc_fgi_data.csv")     # Output 2
FGI_API_URL = "https://api.alternative.me/fng/?limit=0&format=json"
START_DATE_FILTER = '2023-01-01' # <-- CRITICAL FILTER ADDED HERE

def fetch_and_process_fgi_data():
    """Fetches and saves historical Fear & Greed Index data."""
    # (Function remains the same as FGI data is fetched in full and merged later)
    print("⏳ Fetching historical F&G Index data...")

    try:
        response = requests.get(FGI_API_URL, timeout=10)
        response.raise_for_status()

        data = response.json().get('data')
        if not data: return None

        fgi_df = pd.DataFrame(data)
        fgi_df['date'] = pd.to_datetime(fgi_df['timestamp'].astype(int), unit='s')
        fgi_df['value'] = pd.to_numeric(fgi_df['value'], errors='coerce')
        fgi_df = fgi_df[['date', 'value', 'value_classification']].copy()
        fgi_df.set_index('date', inplace=True)
        fgi_df.rename(columns={'value': 'FGI_Score', 'value_classification': 'FGI_Sentiment'}, inplace=True)
        fgi_df.sort_index(inplace=True)

        os.makedirs(SAVE_FOLDER, exist_ok=True)
        fgi_df.to_csv(FGI_PROCESSED_FILE, index=True)
        print(f"✅ FGI data retrieved and saved to {FGI_PROCESSED_FILE}. Total bars: {len(fgi_df)}")
        return fgi_df

    except requests.exceptions.RequestException as e:
        print(f"❌ Error during API request: {e}")
        return None
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
        return None

def load_and_clean_ta_data():
    """Loads, cleans, filters, and saves the Technical Analysis data."""
    try:
        # 1. Load Price and TA Data
        data = pd.read_csv(PROCESSED_FILE, index_col=0, parse_dates=True)

        # 2. Selecting and standardizing core columns
        data_for_bt = data[['Close', 'ATR', 'RSI', 'EMA_20', 'MACD', 'MACD_Signal', 'Close_Lag_1']].copy()
        data_for_bt.columns = ['Close', 'ATR', 'RSI', 'EMA_20', 'MACD', 'MACD_Signal', 'Close_Lag_1']
        data_for_bt.dropna(inplace=True)
        data_for_bt.sort_index(inplace=True)

        # 3. APPLY THE CRITICAL DATE FILTER
        # This limits the TA data to the period we are interested in for backtesting.
        data_for_bt = data_for_bt.loc[START_DATE_FILTER:].copy()

        # 4. SAVE the cleaned DataFrame
        data_for_bt.to_csv(CLEANED_TA_FILE, index=True)

        print(f"✅ Price and TA data loaded and cleaned successfully.")
        print(f"✅ Data filtered starting from {START_DATE_FILTER}.")
        print(f"✅ Cleaned TA data SAVED to: {CLEANED_TA_FILE}. Total usable bars: {len(data_for_bt)}")
        return data_for_bt

    except FileNotFoundError:
        print(f"❌ Error: Processed data file not found at {PROCESSED_FILE}.")
        return None
    except Exception as e:
        print(f"❌ Error during TA data loading and saving: {e}")
        return None

# --- EXECUTION ---
ta_df = load_and_clean_ta_data()
fgi_df = fetch_and_process_fgi_data()

if ta_df is not None and fgi_df is not None:
    print("\n--- Data files ready for merging. ---")

✅ Price and TA data loaded and cleaned successfully.
✅ Data filtered starting from 2023-01-01.
✅ Cleaned TA data SAVED to: /content/drive/MyDrive/ColabNotebooks/Proj5/data_btc/btc_cleaned_ta_data.csv. Total usable bars: 1006
⏳ Fetching historical F&G Index data...
✅ FGI data retrieved and saved to /content/drive/MyDrive/ColabNotebooks/Proj5/data_btc/btc_fgi_data.csv. Total bars: 2831

--- Data files ready for merging. ---


#####<b>Final Merge TA and Sentiment Data</b>

In [ ]:
## -------------------------------------------------------------- ##
#  FINAL MERGE AND GLOBAL DATA PREPARATION
# --- Using the previously confirmed working pd.DataFrame.merge method ---
## -------------------------------------------------------------- ##

# --- CONFIGURATION ---
SAVE_FOLDER = '/content/drive/MyDrive/ColabNotebooks/Proj5/data_btc'
CLEANED_TA_FILE = os.path.join(SAVE_FOLDER, "btc_cleaned_ta_data.csv")
FGI_PROCESSED_FILE = os.path.join(SAVE_FOLDER, "btc_fgi_data.csv")
FINAL_MERGED_FILE = os.path.join(SAVE_FOLDER, "btc_final_merged_data.csv")

# Global variable to hold the final merged data for the strategy
data_for_bt = None

def merge_and_save_final_data():
    """
    Loads CLEANED TA data and FGI data, performs a Left Merge using the Date index,
    fills missing FGI dates conservatively, and saves the final file.
    """
    global data_for_bt

    try:
        # 1. Load CLEANED TA Data (The Master DataFrame)
        data_for_bt_local = pd.read_csv(CLEANED_TA_FILE, index_col=0, parse_dates=True)
        data_for_bt_local.sort_index(inplace=True)

        if data_for_bt_local.empty:
            print(f"❌ Error: Cleaned TA data file at {CLEANED_TA_FILE} is empty.")
            return

        # 2. Load FGI Data
        fgi_df = pd.read_csv(FGI_PROCESSED_FILE, index_col=0, parse_dates=True)
        fgi_df.sort_index(inplace=True)

        # 3. Perform Left Merge using the index (Date)
        # We merge the FGI data onto the TA data (data_for_bt_local).
        merged_df = data_for_bt_local.merge(fgi_df[['FGI_Score', 'FGI_Sentiment']],
                                            left_index=True,  # Match using the index of the left df (TA)
                                            right_index=True, # Match using the index of the right df (FGI)
                                            how='left')       # Keep all TA rows

        # 4. Handle Missing FGI Data (Fill conservatively)
        merged_df['FGI_Score'].fillna(50, inplace=True)
        merged_df['FGI_Sentiment'].fillna('Neutral', inplace=True)

        # 5. Save the Final Combined Data to a new file 💾
        merged_df.to_csv(FINAL_MERGED_FILE, index=True)

        # Update the global variable for the next code cell
        data_for_bt = merged_df

        print(f"✅ TA and Sentiment data merged and saved to {FINAL_MERGED_FILE}.")
        print(f"   Total bars after merge: {len(data_for_bt)}")
        print(f"   Date range: {data_for_bt.index.min().date()} to {data_for_bt.index.max().date()}")

        # Display the head and tail for confirmation
        print("\n--- Merged Data Head (2023 History Confirmed) ---")
        print(data_for_bt[['Close', 'RSI', 'FGI_Score', 'FGI_Sentiment']].head())
        print("\n--- Merged Data Tail (Recent FGI Scores Confirmed) ---")
        print(data_for_bt[['Close', 'RSI', 'FGI_Score', 'FGI_Sentiment']].tail())


    except FileNotFoundError as e:
        print(f"❌ Error: Required file not found. Check if '{e.filename}' exists.")
        print("Ensure Block 1 (TA and FGI preparation) was run and saved the files correctly.")
    except KeyError as e:
        print(f"❌ FINAL ERROR: Column {e} not found. Please check column names in the FGI CSV file.")
    except Exception as e:
        print(f"❌ An unexpected error occurred during data merging and saving: {e}")

# --- EXECUTION ---
merge_and_save_final_data()

✅ TA and Sentiment data merged and saved to /content/drive/MyDrive/ColabNotebooks/Proj5/data_btc/btc_final_merged_data.csv.
   Total bars after merge: 1006
   Date range: 2023-02-03 to 2025-11-05

--- Merged Data Head (2023 History Confirmed) ---
                   Close        RSI  FGI_Score FGI_Sentiment
Date                                                        
2023-02-03  23449.322266  71.088891       60.0         Greed
2023-02-04  23331.847656  69.238110       58.0         Greed
2023-02-05  22955.666016  63.533881       58.0         Greed
2023-02-06  22760.109375  60.732744       56.0         Greed
2023-02-07  23264.291016  65.015328       54.0       Neutral

--- Merged Data Tail (Recent FGI Scores Confirmed) ---
                    Close        RSI  FGI_Score FGI_Sentiment
Date                                                         
2025-10-31  109556.164062  44.513921       29.0          Fear
2025-11-01  110064.015625  45.727378       33.0          Fear
2025-11-02  110639.625

##### <b>Rule-Based Adaptive Multiplier</b>
Source: https://www.youtube.com/watch?v=nzpIhdvDyyo

In [ ]:
## ---------------------------------------------------------------- ##
# SIMPLIFIED RULE-BASED ADAPTIVE MULTIPLIER
# --- Focuses only on MACD Trend and FGI Extremes ---
## ---------------------------------------------------------------- ##

def get_rule_based_multiplier(current_data: pd.Series, combined_action: str) -> float:
    """
    Generates a simplified contextual risk multiplier (0.0 to 1.5) by checking
    if MACD trend and FGI extremes confirm the combined trading action.
    """

    rsi = current_data['RSI']
    macd_delta = current_data['MACD'] - current_data['MACD_Signal']
    fgi_score = current_data['FGI_Score']

    # Base Multiplier: Neutral starting point
    multiplier = 1.0

    # Check if the core action is a BUY/LONG
    is_long_action = 'BUY' in combined_action.upper() or 'LONG' in combined_action.upper()

    # If the strategy is FLAT, exit immediately
    if 'FLAT' in combined_action.upper() or 'HOLD' in combined_action.upper():
        return 1.0

    # --- 1. MACD Trend Confirmation (Adds/Subtracts 0.2) ---
    # The MACD delta should align with the trade direction.
    if (is_long_action and macd_delta > 0) or (not is_long_action and macd_delta < 0):
        # Trend Confirms Action
        multiplier += 0.2
    else:
        # Trend Contradicts Action (Trading against momentum is riskier)
        multiplier -= 0.2

    # --- 2. FGI Extreme Sentiment (Adds/Subtracts 0.3) ---
    # We look for contrarian signals (Fear for Buys, Greed for Sells)
    if fgi_score <= 30: # Extreme Fear (Contrarian Buy Signal)
        multiplier += (0.3 if is_long_action else -0.3)
    elif fgi_score >= 70: # Extreme Greed (Contrarian Sell Signal)
        multiplier += (-0.3 if is_long_action else 0.3)

    # --- 3. RSI Over-extension Filter (Adds/Subtracts 0.1) ---
    # Punish the trade if momentum is exhausted (RSI is too high for a Buy, too low for a Sell)
    if (is_long_action and rsi >= 75) or (not is_long_action and rsi <= 25):
        multiplier -= 0.1
    elif (is_long_action and rsi <= 30) or (not is_long_action and rsi >= 70):
        multiplier += 0.1 # Reward entering at oversold/overbought extremes

    # 4. Clamp the output to ensure it stays within the required range
    final_multiplier = max(0.0, min(1.5, multiplier))

    return final_multiplier

# Assign the rule-based function to the name expected by Block 4
get_final_multiplier = get_rule_based_multiplier

# print("\n--- Simplified Rule-Based Adaptive Multiplier Function Defined ---")
# print("This function is now stable and ready for integration into the strategy.")

##### <b> Trading Global Configuration </b>

In [ ]:
# --- GLOBAL SYSTEM CONFIGURATION ---
import pandas as pd
import numpy as np
import os
import json
import sys

# Financial Parameters
STARTING_BUDGET = 100000.0 # Configurable initial budget
COMMISSION_RATE = 0.002
RISK_FREE_RATE_ANNUAL = 0.04
DCA_AMOUNT_DAILY = 100.0   # Base daily accumulation amount
ATR_MULTIPLIER = 3.0       # Stop-loss risk factor for tactical trades

# Strategy Parameters (TA-Based Rules)
RSI_OVERSOLD_THRESHOLD = 30 # Threshold for potential "Value/Range" accumulation
RSI_OVERBOUGHT_THRESHOLD = 70 # Threshold for potential "Swing" profit-taking
SWING_TRADE_ALLOCATION_MAX = 0.60 # Max percentage of budget for active trades

##### <b> Dynamic Trading Strategy </b>

In [ ]:
## -------------------------------------------------------------- ##
# ----- CORE EXECUTION FUNCTIONS (MODIFIES PORTFOLIO) -----
## -------------------------------------------------------------- ##
import pandas as pd
# Assume global variables (DCA_AMOUNT_DAILY, ATR_MULTIPLIER, COMMISSION_RATE, etc.) are available

def execute_dca_buy(row, portfolio: dict) -> bool:
    """Executes a daily DCA buy and modifies the portfolio state."""
    global DCA_AMOUNT_DAILY, COMMISSION_RATE, RSI_OVERSOLD_THRESHOLD

    current_close = row['Close']
    dca_amount = DCA_AMOUNT_DAILY

    # Conditional DCA: Increase buy size during accumulation periods
    if row['RSI'] < RSI_OVERSOLD_THRESHOLD:
        dca_amount *= 1.5

    if portfolio['cash'] >= dca_amount:
        commission = dca_amount * COMMISSION_RATE
        buy_qty = (dca_amount - commission) / current_close

        # Update portfolio directly
        portfolio['cash'] -= dca_amount
        portfolio['btc_qty'] += buy_qty
        return True
    return False


def open_tactical_trade(row, portfolio: dict, allocation_pct: float) -> bool:
    """Opens a tactical position and modifies the portfolio state."""
    global ATR_MULTIPLIER, COMMISSION_RATE

    if portfolio.get('swing_qty', 0.0) > 0:
        return False

    trade_budget = portfolio['cash'] * allocation_pct

    if trade_budget > 0:
        entry_price = row['Close']
        commission = trade_budget * COMMISSION_RATE
        buy_qty = (trade_budget - commission) / entry_price

        # ATR Stop Loss Calculation
        stop_loss_level = entry_price - (row['ATR'] * ATR_MULTIPLIER)

        # Update portfolio directly
        portfolio['cash'] -= trade_budget
        portfolio['btc_qty'] += buy_qty
        portfolio['swing_qty'] = buy_qty
        portfolio['swing_entry_price'] = entry_price
        portfolio['stop_loss_level'] = stop_loss_level
        return True
    return False


def close_tactical_trade(row, portfolio: dict, final_action: str) -> bool:
    """Determines and executes the parameters for closing the tactical trade (SL or TP)."""
    global COMMISSION_RATE

    if portfolio.get('swing_qty', 0.0) == 0.0:
        return False

    btc_to_sell = portfolio['swing_qty']
    current_close = row['Close']
    entry_price = portfolio['swing_entry_price']

    # Condition 1: STOP-LOSS CHECK (Risk Management Override)
    if current_close < portfolio['stop_loss_level']:
        exit_price = portfolio['stop_loss_level']

    # Condition 2: PROFIT-TAKE CHECK (Strategy Driven)
    elif current_close >= entry_price * 1.05 and \
      ('BUY' not in final_action): # If profitable AND the new day's signal is not a strong BUY/LONG
      exit_price = current_close

    else:
        return False # No exit condition met

    # Execute Sale
    sale_usd = btc_to_sell * exit_price
    commission = sale_usd * COMMISSION_RATE

    # Update portfolio directly
    portfolio['cash'] += sale_usd - commission
    portfolio['btc_qty'] -= btc_to_sell
    portfolio['swing_qty'] = 0.0
    portfolio['stop_loss_level'] = 0.0

    return True

# print("✅ Execution functions updated to modify the portfolio state directly.")

##### <b>Combined Strategy Decision Engine</b>

In [ ]:
## -------------------------------------------------------------- ##
# COMBINED STRATEGY (MASTER DECIDER AND EXECUTOR)
# --- Updated: Removed dependency on 'decide_strategy' and integrated TA logic ---
## -------------------------------------------------------------- ##


def get_combined_signal_and_execute(current_data: pd.Series, portfolio: dict) -> dict:
    """
    Combines TA (in-line) and Sentiment, applies the Adaptive Multiplier,
    makes the final decision, and executes the trade by modifying the portfolio.
    """
    global SWING_TRADE_ALLOCATION_MAX

    # --- 1. BASE DECISION LOGIC (TA and Sentiment Score Calculation) ---

    # 1.1 Calculate TA Mode In-Line (REPLACEMENT FOR decide_strategy)
    rsi = current_data['RSI']
    macd_delta = current_data['MACD'] - current_data['MACD_Signal']

    if macd_delta > 0 and rsi > 55:
        # Strong Momentum, suggests swinging
        ta_mode = 'SWING_TRADE'
    elif rsi < 50 and rsi > 30 and abs(macd_delta) < 0.1:
        # Low momentum, likely consolidation/range
        ta_mode = 'RANGE_BOUND'
    else:
        ta_mode = 'NEUTRAL'

    # 1.2 Calculate Scores
    fgi_score = current_data['FGI_Score']

    sentiment_score = 0
    if fgi_score <= 24: sentiment_score = 2
    elif fgi_score <= 49: sentiment_score = 1
    elif fgi_score >= 75: sentiment_score = -1

    ta_score = 0
    if ta_mode == 'SWING_TRADE': ta_score = 2
    elif ta_mode == 'RANGE_BOUND': ta_score = 1
    # NEUTRAL ta_mode implicitly gives ta_score = 0

    combined_score = ta_score + sentiment_score

    # 1.3 Determine Final Action and Base Allocation
    if combined_score >= 4:
        final_action = "AGGRESSIVE_BUY"
        base_allocation_pct = SWING_TRADE_ALLOCATION_MAX # Base 100% of max
    elif combined_score >= 2:
        final_action = "MODERATE_BUY"
        base_allocation_pct = SWING_TRADE_ALLOCATION_MAX * 0.5 # Base 50% of max
    elif combined_score <= -1:
        final_action = "AVOID_ENTRY"
        base_allocation_pct = 0.0
    else:
        final_action = "HOLD_DCA_ONLY"
        base_allocation_pct = 0.0

    # --- 2. MULTIPLIER CALCULATION AND APPLICATION ---

    # Only calculate and apply the multiplier if there is a potential BUY action
    if 'BUY' in final_action:
        # Get the adaptive risk multiplier from the stable rule-based function
        risk_multiplier = get_final_multiplier(current_data, final_action)

        # Apply the multiplier to the base trade size
        allocation_pct = base_allocation_pct * risk_multiplier

        # Final safety clamp: Ensure we don't exceed the global max allocation
        allocation_pct = min(allocation_pct, SWING_TRADE_ALLOCATION_MAX)

        # If the multiplier drops the trade size too low, update the action description
        if allocation_pct < SWING_TRADE_ALLOCATION_MAX * 0.1:
             final_action = f"RISK_BLOCKED_{final_action}"
    else:
        allocation_pct = base_allocation_pct
        risk_multiplier = 1.0 # For reporting purposes

    # --- 3. EXECUTION LOGIC ---
    trade_occurred = False

    # Use a variable for the action string to pass to execution functions
    current_final_action = final_action

    # A. Execute Exits (Must check this first)
    if close_tactical_trade(current_data, portfolio, current_final_action):
        trade_occurred = True

    # B. Execute DCA Buy (Usually runs regardless of tactical signal)
    if execute_dca_buy(current_data, portfolio):
        trade_occurred = True

    # C. Execute TACTICAL Entry (Driven by the final_action and calculated allocation_pct)
    if 'BUY' in current_final_action and portfolio['swing_qty'] == 0.0 and allocation_pct > 0:
        if open_tactical_trade(current_data, portfolio, allocation_pct):
            trade_occurred = True

    return {
        'final_action': final_action,
        'trade_occurred': trade_occurred,
        'ta_mode': ta_mode,
        'fgi_score': fgi_score,
        'multiplier': risk_multiplier,
        'final_allocation_pct': allocation_pct
    }

# print("✅ Block 4: Combined strategy function successfully updated. Dependency on 'decide_strategy' is removed.")

##### <b> Full Strategy Sanity Check </b>

In [ ]:
## ---------------------------------------------------------------- ##
#     FULL STRATEGY SANITY CHECK (LAST 120 DAYS)
# --- Verifies the combined decision, sentiment, and multiplier logic ---
## ---------------------------------------------------------------- ##
import pandas as pd
import sys

# Global variable data_for_bt is assumed to be loaded and prepared from Cell 1.2
# Strategy functions (decide_strategy, get_combined_signal_and_execute, etc.) are assumed to be defined.

LOOKBACK_DAYS = 120

# 1. Prepare Data: Select the last 120 bars
sanity_check_data = data_for_bt.tail(LOOKBACK_DAYS).copy() # Use .copy() for safety

if sanity_check_data.empty or len(sanity_check_data) < LOOKBACK_DAYS:
    print(f"❌ Error: Insufficient data. Need at least {LOOKBACK_DAYS} days.")
    sys.exit(1)

# 2. Apply the get_combined_signal_and_execute function to every row
# We will use a dummy portfolio as the function requires one,
# but the portfolio state won't persist across days in this sanity check loop.
strategy_results = []
# Initialize a dummy portfolio that doesn't change state during this check
DUMMY_PORTFOLIO = {
    'cash': 100000.0,
    'btc_qty': 0.0,
    'swing_qty': 0.0,
    'swing_entry_price': 0.0,
    'stop_loss_level': 0.0,
}

print("Running full combined strategy check on the last 120 days...")
for index, row in sanity_check_data.iterrows():
    # Use the master execution function (read-only mode)
    result = get_combined_signal_and_execute(row, DUMMY_PORTFOLIO.copy())

    # Store relevant metrics from the result dictionary
    strategy_results.append({
        'Date': index,
        'Close': row['Close'],
        'TA_Mode': result['ta_mode'],
        'FGI_Score': result['fgi_score'],
        'Final_Action': result['final_action'],
        'Risk_Multiplier': result['multiplier'],
        'Final_Allocation_Pct': result['final_allocation_pct'],
    })

results_df = pd.DataFrame(strategy_results).set_index('Date')


# 3. ANALYSIS AND OUTPUT
print("\n=========================================================")
print(f" Full Adaptive Strategy Sanity Check: Last {LOOKBACK_DAYS} Days")
print("=========================================================")

# A. Final Action Frequency (Strategy Bias)
print("\n--- Final Action Frequency (Market View) ---")
print("Shows how often the combined system decided on each action.")
action_frequency = results_df['Final_Action'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
print(action_frequency)

# B. Risk Multiplier Impact
print("\n--- Risk Assessment Impact ---")
print(f"Price Change over {LOOKBACK_DAYS} days: {((results_df['Close'].iloc[-1] / results_df['Close'].iloc[0]) - 1) * 100:.2f}%")
print(f"Average FGI Score: {results_df['FGI_Score'].mean():.1f}")
print(f"Average Risk Multiplier: {results_df['Risk_Multiplier'].mean():.3f} (Neutral is 1.0)")
print(f"Avg. Final Allocation (Adjusted): {results_df['Final_Allocation_Pct'].mean() * 100:.2f}%")


# C. Current Day Decision (Immediate Action)
last_day = results_df.iloc[-1]
print("\n--- Today's Full Strategy Decision ---")
print(f"Date: {last_day.name.strftime('%Y-%m-%d')}")
print(f"TA Mode: {last_day['TA_Mode']}")
print(f"FGI Score: {last_day['FGI_Score']:.0f}")
print(f"Final Action: {last_day['Final_Action']}")
print(f"Multiplier Applied: {last_day['Risk_Multiplier']:.3f}")
print(f"Final Entry Allocation: {last_day['Final_Allocation_Pct'] * 100:.1f}%")

Running full combined strategy check on the last 120 days...

 Full Adaptive Strategy Sanity Check: Last 120 Days

--- Final Action Frequency (Market View) ---
Shows how often the combined system decided on each action.
Final_Action
MODERATE_BUY       65.0%
HOLD_DCA_ONLY     31.67%
AVOID_ENTRY        1.67%
AGGRESSIVE_BUY     1.67%
Name: proportion, dtype: object

--- Risk Assessment Impact ---
Price Change over 120 days: -4.81%
Average FGI Score: 53.8
Average Risk Multiplier: 1.010 (Neutral is 1.0)
Avg. Final Allocation (Adjusted): 20.70%

--- Today's Full Strategy Decision ---
Date: 2025-11-05
TA Mode: NEUTRAL
FGI Score: 23
Final Action: MODERATE_BUY
Multiplier Applied: 1.100
Final Entry Allocation: 33.0%


#### <b> Backtesting </b>
<b> Backtesting 1: </b> In-Sample (IS) - period from 2023-11-01 to 2024-10-31<br>
 <b> Backtesting 2: </b> Out-of-Sample (OOS) - period from 2024-11-01 to 2025-10-31

In [ ]:
## ---------------------------------------------------------------- ##
# --- FINAL BACKTESTING ENGINE (SIMPLE CALL) ---
## ---------------------------------------------------------------- ##

# Assume global variables: STARTING_BUDGET, RISK_FREE_RATE_ANNUAL
# Assume file path: FINAL_MERGED_FILE

def run_simplified_backtest(start_date_str, end_date_str):
    """
    Runs the backtest simulation by only calling the combined strategy function.
    """
    global STARTING_BUDGET, RISK_FREE_RATE_ANNUAL

    # 1. Load Correct Data
    try:
        data_for_bt_local = pd.read_csv(FINAL_MERGED_FILE, index_col=0, parse_dates=True)
        data_sim = data_for_bt_local.loc[start_date_str:end_date_str].copy()
    except FileNotFoundError:
        print(f"❌ Error: Final merged data file not found at {FINAL_MERGED_FILE}. Cannot proceed.")
        return None
    except Exception:
        print(f"❌ Error: Date range {start_date_str} to {end_date_str} failed to slice.")
        return None

    if len(data_sim) <= 1:
        print(f"❌ Data slice for {start_date_str} to {end_date_str} is too short.")
        return None

    print("\n" + "="*80)
    print(f" BACKTEST: {start_date_str} to {end_date_str}")
    print("="*80)

    # 2. Initialize Portfolio State and Metrics
    portfolio = {
        'cash': STARTING_BUDGET,
        'btc_qty': 0.0,
        'swing_qty': 0.0,
        'swing_entry_price': 0.0,
        'stop_loss_level': 0.0,
    }
    portfolio_value_history = [STARTING_BUDGET]
    total_trades = 0

    # 3. Main Execution Loop
    for date, row in data_sim.iloc[1:].iterrows():
        current_close = row['Close']

        # --- THE ONLY CALL REQUIRED ---
        signal_results = get_combined_signal_and_execute(row, portfolio)

        if signal_results['trade_occurred']:
            total_trades += 1

        # Log Performance
        total_btc_value = portfolio['btc_qty'] * current_close
        current_total_value = portfolio['cash'] + total_btc_value
        portfolio_value_history.append(current_total_value)

    # 4. Final Metrics Calculation (Streamlined)
    port_series = pd.Series(portfolio_value_history, index=data_sim.index)

    final_value = port_series.iloc[-1]
    total_return_pct = ((final_value - STARTING_BUDGET) / STARTING_BUDGET) * 100

    cumulative_max = port_series.cummax()
    drawdown = (port_series - cumulative_max) / cumulative_max
    max_drawdown_pct = abs(drawdown.min()) * 100

    daily_returns = port_series.pct_change().dropna()
    sharpe_ratio = 0.0
    if daily_returns.std() > 0:
        daily_risk_free_rate = RISK_FREE_RATE_ANNUAL / 365
        sharpe_ratio = (daily_returns.mean() - daily_risk_free_rate) / daily_returns.std() * np.sqrt(365)

    duration = data_sim.index.max() - data_sim.index.min()

    # 5. Print Results
    print(f"\nRESULTS: Combined TA + Sentiment")
    print(f"Start:                 {data_sim.index.min()}")
    print(f"End:                   {data_sim.index.max()}")
    print(f"Duration:              {duration}")
    print("-" * 35)
    print(f"Return [%]:            {total_return_pct:.2f}")
    print(f"Max. Drawdown [%]:     {-max_drawdown_pct:.2f}")
    print(f"Sharpe Ratio:          {sharpe_ratio:.4f}")
    print(f"Total Trades:          {total_trades}")
    print("=" * 80)

    return port_series

# --- EXECUTION ---
# Backtesting 1: period from 2023/11/01 to 2024-10-31
performance_bt1 = run_simplified_backtest("2023-11-01", "2024-10-31")

# Backtesting 2: period from 2024-11-01 to 2025-10-31
performance_bt2 = run_simplified_backtest("2024-11-01", "2025-10-31")


 BACKTEST: 2023-11-01 to 2024-10-31

RESULTS: Combined TA + Sentiment
Start:                 2023-11-01 00:00:00
End:                   2024-10-31 00:00:00
Duration:              365 days 00:00:00
-----------------------------------
Return [%]:            17.14
Max. Drawdown [%]:     -14.54
Sharpe Ratio:          0.7535
Total Trades:          365

 BACKTEST: 2024-11-01 to 2025-10-31

RESULTS: Combined TA + Sentiment
Start:                 2024-11-01 00:00:00
End:                   2025-10-31 00:00:00
Duration:              364 days 00:00:00
-----------------------------------
Return [%]:            13.53
Max. Drawdown [%]:     -8.79
Sharpe Ratio:          0.6241
Total Trades:          364


###### <b>Conclusion:</b> The combined TA + Sentiment strategy appears to be a strong performer in bullish, high-momentum environments (BT1) where trend-following and contrarian buys are effective. However, its effectiveness drops substantially in challenging, range-bound, or corrective markets (BT2), where it takes more risk for smaller gains.